In [ ]:
day_obs = 20260218

# Diasources and Visits 

Query the number of diaSources reported in `lsst.prompt.prod.numDiaSourcesGood` and merge with visit information.

In [ ]:
import os
from astropy.time import Time, TimeDelta
from astropy.coordinates import EarthLocation
from IPython.display import display, HTML
from rubin_nights import connections
from rubin_nights.influx_query import InfluxQueryClient
import rubin_nights.dayobs_utils as rn_dayobs

import numpy as np
import pandas as pd 

import rubin_sim.maf as maf
from lsst_survey_sim import plot

import matplotlib.pyplot as plt

import rubin_nights.plot_utils as rn_plots
colors = rn_plots.PlotStyles.band_colors
markers = rn_plots.PlotStyles.band_symbols

import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

In [ ]:
# Where is the notebook running? (RSPs are 'special')
current_location = os.getenv("EXTERNAL_INSTANCE_URL", "")

# RUBIN_SIM_DATA_DIR at usdf
if 'usdf' in current_location:
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

endpoints = connections.get_clients()

In [ ]:
# Single night
tstart, tend = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)

# OR .. 
#tstart = Time("2025-10-24T12:00:00")
#tend = Time.now()

sasquatch = InfluxQueryClient("usdfdev", db_name="lsst.prompt")

# DiaSources
dia_det = sasquatch.select_time_series("lsst.prompt.prod.numDiaSourcesGood", 
                                  ["visit", "band", "day_obs", "detector", "numAllDiaSources", "numGoodDiaSources", "run"], 
                                  tstart, tend)

if len(dia_det) > 0:
    alertsum = dia_det.groupby("visit").agg({'numAllDiaSources': 'sum', 'numGoodDiaSources': ('sum', 'median'), 'detector': 'count'})
    alertsum.index = alertsum.index.astype(int)
    cols = [f"{c[0]}_{c[1]}" for c in alertsum.columns]
    alertsum = alertsum.droplevel(level=0, axis=1)
    alertsum.columns = cols
    alertsum.rename({"detector_count": "nDiaDetectors_count"}, axis=1, inplace=True)
else:
    alertsum = pd.DataFrame([], columns=['numAllDiaSources_sum', 'numGoodDiaSources_sum', 'numGoodDiaSources_median', 'nDiaDetectors_count'])
    print("no alerts")

# Solar system alerts
sso_det = sasquatch.select_time_series("lsst.prompt.prod.numSsObjects", ["visit", "band", "day_obs", "detector", "NumSsObjectsMetric", "run"], tstart, tend)

if len(sso_det) > 0:
    ssosum = sso_det.groupby("visit").agg({'NumSsObjectsMetric': 'sum', 'detector': 'count'})
    ssosum.index = ssosum.index.astype(int)
    ssosum.rename({"detector": "nSsDetectors_count", "NumSsObjectsMetric": "numSsObjects_sum"}, axis=1, inplace=True)
else:
    ssosum = pd.DataFrame([], columns=['numSsObjects_sum', 'nSsDetectors_count'])
    print("no sso alerts")

# And direct solar system associations (not alerts)
asso_det = sasquatch.select_time_series("lsst.prompt.prod.numDirectSsObjects", ["visit", "band", "day_obs", "detector", "NumSsObjectsMetric", "run"], tstart, tend)

if len(asso_det) > 0:
    assosum = asso_det.groupby("visit").agg({'NumSsObjectsMetric': 'sum', 'detector': 'count'})
    assosum.index = assosum.index.astype(int)
    assosum.rename({"detector": "nDSsDetectors_count", "NumSsObjectsMetric": "numDirectSsObjects_sum"}, axis=1, inplace=True)
else:
    assosum = pd.DataFrame([], columns=['numDirectSsObjects_sum', 'nDSsDetectors_count'])
    print("no direct sso associations")

In [ ]:
alertsum = pd.merge(ssosum, alertsum, how='outer', left_index=True, right_index=True)
allsum = pd.merge(assosum, alertsum, how='outer', left_index=True, right_index=True)

In [ ]:
# fields = ['visit', 'exposure', 'detector', 'day_obs', 'ra', 'dec', 'band', 'nPsfStar', 'psfArea', 'psfSigma', 'skyBg', 'skyNoise', 'zeroPoint']
# tt2 = sasquatch.select_time_series("lsst.prompt.prod.initialPviSummaryMetrics", fields, tstart, tend)

In [ ]:
from rubin_nights.reference_values import SCIENCE_PROGRAMS

programs = SCIENCE_PROGRAMS
constraint = " or ".join([f"science_program = '{p}'" for p in programs])
visits = endpoints['consdb_tap'].get_visits("lsstcam", tstart, tend, visit_constraint=constraint)

In [ ]:
print("number of visits:", len(visits))
print("number of visits with numGoodDiaSources", len(alertsum))
print("number of visits with associated SSOs:", len(assosum))
print("number of good diasources -- note this is not quite the number of alerts, as alerts are not 'numGoodDiaSources' exactly:", alertsum.numGoodDiaSources_sum.sum())
print("(generally there are more alerts than goodDiaSources)")

In [ ]:
if len(visits) > 0 and len(alertsum) > 0:
    vv = pd.merge(visits, allsum, left_on='visit_id', right_index=True, how='outer')
    cols = ['visit_id', 'observation_reason', 'target_name', 'obs_start', 'band', 's_ra', 's_dec', 'sky_rotation', 'clouds', 'fwhm_eff', 'cat_m5', 
            'numGoodDiaSources_sum', 'numGoodDiaSources_median', 'nDiaDetectors_count', 
            'numSsObjects_sum', 'nSsDetectors_count', 'numDirectSsObjects_sum', 'nDSsDetectors_count']

In [ ]:
#vv[cols]
#vv.groupby("science_program").agg({'visit_id': 'count'})
#vv.query("numGoodDiaSources_sum > 0").sort_values(by="numGoodDiaSources_sum")[cols]

In [ ]:
lsst_site = EarthLocation.of_site("rubin")
mean_lst = Time(tstart + (tend - tstart)/2, location=lsst_site).sidereal_time("mean")
lon_0 = mean_lst.deg

In [ ]:
run_calc = True
# all kinds of detections including direct sso objects?
#q = vv.query("numGoodDiaSources_sum > 0 or numDirectSsObjects_sum > 0")
# or just alerts
#q = vv.query("numGoodDiaSources_sum > 0")
# or all visits
q = vv
if run_calc and len(q) > 0:

    nvisits_v = {}
    m_nvis = maf.CountMetric(col='obs_start_mjd', metric_name = "Nvisits")
    s = maf.HealpixSlicer(nside=64, lon_col='s_ra', lat_col='s_dec', rot_sky_pos_col_name = 'sky_rotation')
    for b in ['u', 'g', 'r', 'i', 'z', 'y', 'all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = q.to_records()
        else:
            opsvis = q.query("band == @b").to_records()
        nvisits_v[b] = maf.MetricBundle(m_nvis, s, constraint)
        g = maf.MetricBundleGroup({f'nvisits {b}': nvisits_v[b],}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)    

    background = plot.get_background(nside=64)
    
    fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 10),)
    axdict = {"u": ax[0][0], "g": ax[0][1], "r": ax[0][2],
              "i": ax[1][0], "z": ax[1][1], "y": ax[1][2], "all": None}
    for b in ["u", "g", "r", "i", "z", "y"]:
        if nvisits_v[b].metric_values is not None:
            if len(nvisits_v[b].metric_values.compressed()) > 1:
                vmax = np.percentile(nvisits_v[b].metric_values.compressed(), 95)
            else:
                vmax = None
            label_dec = False
            if b == 'u' or b == 'i':
                label_dec = True
            fig = plot.make_plot(nvisits_v[b], background=background, proj='McBryde', lon_0=lon_0, vmax=vmax, ax=axdict[b], title=f"LSSTCam band visits {b}", label_dec=label_dec)
    fig.tight_layout()
    
    vmax = np.percentile(nvisits_v['all'].metric_values.compressed(), 95)
    fig = plot.make_plot(nvisits_v['all'], background=background, proj='mcbryde', lon_0=lon_0, vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")

In [ ]:
run_calc = True
# all kinds of detections including direct sso objects?
#q = vv.query("numGoodDiaSources_sum > 0 or numDirectSsObjects_sum > 0")
# or just alerts
q = vv.query("numGoodDiaSources_sum > 0")
if run_calc and len(q) > 0:
    nvisits = {}
    m_nvis = maf.CountMetric(col='obs_start_mjd', metric_name = "Nvisits")
    s = maf.HealpixSlicer(nside=64, lon_col='s_ra', lat_col='s_dec', rot_sky_pos_col_name = 'sky_rotation')
    for b in ['u', 'g', 'r', 'i', 'z', 'y', 'all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = q.to_records()
        else:
            opsvis = q.query("band == @b").to_records()
        nvisits[b] = maf.MetricBundle(m_nvis, s, constraint)
        g = maf.MetricBundleGroup({f'nvisits {b}': nvisits[b],}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)    

    background = plot.get_background(nside=64)
    
    fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 10),)
    axdict = {"u": ax[0][0], "g": ax[0][1], "r": ax[0][2],
              "i": ax[1][0], "z": ax[1][1], "y": ax[1][2], "all": None}
    for b in ["u", "g", "r", "i", "z", "y"]:
        if nvisits[b].metric_values is not None:
            if len(nvisits[b].metric_values.compressed()) > 1:
                vmax = np.percentile(nvisits[b].metric_values.compressed(), 95)
            else:
                vmax = None
            label_dec = False
            if b == 'u' or b == 'i':
                label_dec = True
            fig = plot.make_plot(nvisits[b], background=background, proj='McBryde', lon_0=lon_0, vmax=vmax, ax=axdict[b], title=f"LSSTCam band with Alerts {b}", label_dec=label_dec)
    fig.tight_layout()
    
    vmax = np.percentile(nvisits['all'].metric_values.compressed(), 95)
    fig = plot.make_plot(nvisits['all'], background=background, proj='mcbryde', lon_0=lon_0, vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits with Alerts")

In [ ]:
run_calc = True
# all kinds of detections including direct sso objects?
#q = vv.query("numGoodDiaSources_sum > 0 or numDirectSsObjects_sum > 0")
# or just direct associations of ssos
q = vv.query("numDirectSsObjects_sum > 0")
if run_calc and len(q) > 0:

    nvisits_s = {}
    m_nvis = maf.CountMetric(col='obs_start_mjd', metric_name = "Nvisits")
    s = maf.HealpixSlicer(nside=64, lon_col='s_ra', lat_col='s_dec', rot_sky_pos_col_name = 'sky_rotation')
    for b in ['u', 'g', 'r', 'i', 'z', 'y', 'all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = q.to_records()
        else:
            opsvis = q.query("band == @b").to_records()
        nvisits_s[b] = maf.MetricBundle(m_nvis, s, constraint)
        g = maf.MetricBundleGroup({f'nvisits {b}': nvisits_s[b],}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)    

    background = plot.get_background(nside=64)
    
    fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 10),)
    axdict = {"u": ax[0][0], "g": ax[0][1], "r": ax[0][2],
              "i": ax[1][0], "z": ax[1][1], "y": ax[1][2], "all": None}
    for b in ["u", "g", "r", "i", "z", "y"]:
        if nvisits_s[b].metric_values is not None:
            if len(nvisits_s[b].metric_values.compressed()) > 1:
                vmax = np.percentile(nvisits_s[b].metric_values.compressed(), 95)
            else:
                vmax = None
            label_dec = False
            if b == 'u' or b == 'i':
                label_dec = True
            fig = plot.make_plot(nvisits_s[b], background=background, proj='McBryde', lon_0=lon_0, vmax=vmax, ax=axdict[b], title=f"LSSTCam band with DirectSSOs {b}", label_dec=label_dec)
    fig.tight_layout()
    
    vmax = np.percentile(nvisits_s['all'].metric_values.compressed(), 95)
    fig = plot.make_plot(nvisits_s['all'], background=background, proj='mcbryde', lon_0=lon_0, vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits with DirectSSOs")

In [ ]:
if len(vv) > 0:
    qqq = vv
    fig, ax = plt.subplots(1, 2, figsize=(18, 8))
    subs = ['ddf not cosmos', 'ddf cosmos', 'too', 'twilight_near_sun', 'other' ]
    exclude = np.array([])
    for tt in subs:
        if tt == 'ddf not cosmos':
            qq = qqq.query("target_name.str.contains('ddf_') and ~target_name.str.contains('cosmos')")
            exclude = np.concat([exclude, qq.index])
            marker = 's'
        elif tt == 'ddf cosmos':
            qq = qqq.query("target_name.str.contains('cosmos')")
            exclude = np.concat([exclude, qq.index])
            marker = 'v'
        elif tt == 'too':
            qq = qqq.query("observation_reason.str.contains('too')")
            exclude = np.concat([exclude, qq.index])
            marker = '*'
        elif tt == 'twilight_near_sun':
            qq = qqq.query("observation_reason.str.contains('twilight_near_sun')")
            exclude = np.concat([exclude, qq.index])
            marker = '>'
        else:
            qq = qqq.query("index not in @exclude")
            marker = 'o'
        
        for b in qq.band.unique():
            q = qq.query("band == @b and numGoodDiaSources_sum > 0")
            #print(tt, b, len(q))
            if len(q) > 0:
                if not np.all(np.isnan(q.cat_m5)) and not np.all(np.isnan(q.numGoodDiaSources_sum)):
                    scatter1 = ax[0].scatter(q.cat_m5, q.numGoodDiaSources_sum, s=q.nDiaDetectors_count/189*20 + 5,
                                            marker=marker, color=colors[b], linestyle='', label=f"{b} {tt}")
                    scatter2 = ax[1].scatter(q.nDiaDetectors_count, q.numGoodDiaSources_sum, s=(q.cat_m5 - q.cat_m5.min())*5 + 9, #color=colors[b], 
                                              marker=marker, color=colors[b], linestyle='', label=f"{b} {tt}")

    # ax[0].legend()
    ax[0].grid(alpha=0.3)
    ax[0].set_xlabel("estimated limiting magnitude", fontsize='large')
    ax[0].set_ylabel("nGoodDiaSources per visit", fontsize='large')
    day_title = vv.day_obs.unique()
    if len(day_title) == 1:
        day_title = day_title[0]
    else:
        day_title = f"{day_title.min()} - {day_title.max()}"
    ax[0].set_title(f"Dayobs {day_title}", fontsize='large')
    
    ax[1].legend(loc=(1.01, 0.1))
    ax[1].grid(alpha=0.3)
    ax[1].set_xlabel("nDiaDetectors per visit")
    ax[1].set_ylabel("nGoodDiaSources per visit", fontsize='large')
    ax[1].set_title(f"Dayobs {day_title}", fontsize='large')

In [ ]:
if len(vv) > 0:
    qqq = vv
    fig, ax = plt.subplots(1, 2, figsize=(18, 8))
    subs = ['ddf not cosmos', 'ddf cosmos', 'too', 'twilight_near_sun', 'other' ]
    exclude = np.array([])
    for tt in subs:
        if tt == 'ddf not cosmos':
            qq = qqq.query("target_name.str.contains('ddf_') and ~target_name.str.contains('cosmos')")
            exclude = np.concat([exclude, qq.index])
            marker = 's'
        elif tt == 'ddf cosmos':
            qq = qqq.query("target_name.str.contains('cosmos')")
            exclude = np.concat([exclude, qq.index])
            marker = 'v'
        elif tt == 'too':
            qq = qqq.query("observation_reason.str.contains('too')")
            exclude = np.concat([exclude, qq.index])
            marker = '*'
        elif tt == 'twilight_near_sun':
            qq = qqq.query("observation_reason.str.contains('twilight_near_sun')")
            exclude = np.concat([exclude, qq.index])
            marker = '>'
        else:
            qq = qqq.query("index not in @exclude")
            marker = 'o'
        
        for b in qq.band.unique():
            q = qq.query("band == @b and numDirectSsObjects_sum > 0")
            #print(tt, b, len(q))
            if len(q) > 0:
                if not np.all(np.isnan(q.cat_m5)) and not np.all(np.isnan(q.numDirectSsObjects_sum)):
                    scatter1 = ax[0].scatter(q.eclip_lat, q.cat_m5,
                                            marker=marker, color=colors[b], linestyle='', label=f"{b} {tt}")
                    scatter2 = ax[1].scatter(q.eclip_lat, q.numDirectSsObjects_sum,
                                            marker=marker, color=colors[b], linestyle='', label=f"{b} {tt}")

    #ax.legend(loc=(1.01, 0.2))
    ax[0].grid(alpha=0.3)
    ax[0].set_xlabel("Ecliptic Latitude (deg)", fontsize='large')
    ax[0].set_ylabel("Estimated m5", fontsize='large')
    day_title = vv.day_obs.unique()
    if len(day_title) == 1:
        day_title = day_title[0]
    else:
        day_title = f"{day_title.min()} - {day_title.max()}"
    ax[0].set_title(f"Dayobs {day_title}", fontsize='large')

    ax[1].legend(loc=(1.01, 0.2))
    ax[1].grid(alpha=0.3)
    ax[1].set_xlabel("Ecliptic Latitude (deg)", fontsize='large')
    ax[1].set_ylabel("numDirectSsObjects_sum per visit", fontsize='large')
    day_title = vv.day_obs.unique()
    if len(day_title) == 1:
        day_title = day_title[0]
    else:
        day_title = f"{day_title.min()} - {day_title.max()}"
    ax[1].set_title(f"Dayobs {day_title}", fontsize='large')

In [ ]:
cols= ['visit_id', 'numGoodDiaSources_sum', 'numDirectSsObjects_sum', 'day_obs', 'band', 'eclip_lat', 'gal_lat', 'cat_m5', 'psf_sigma_median', 'clouds', 
       'target_name', 'observation_reason', 'science_program']
#vv.sort_values('numGoodDiaSources_sum', ascending=False)[cols].head(30)
#vv.sort_values('numDirectSsObjects_sum', ascending=False)[cols].head(30)

In [ ]:
#q = vv.query("day_obs == 20251101")# and numGoodDiaSources_sum>0")
q = vv

fig, ax = plt.subplots(7, 1, figsize=(8, 15), sharex=True)

leg_x = 1.01
leg_y = 0.2
fig.subplots_adjust(hspace=0.1)
i = 0
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.fwhm_eff, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(q.obs_start_mjd, q.fwhm_eff, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y))
ax[i].set_ylabel("fwhm (arcsecond)")
ax[i].grid(alpha=0.3)
i += 1
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.clouds, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(q.obs_start_mjd, q.clouds, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y))
ax[i].set_ylabel("cloud extinction")
ax[i].grid(alpha=0.3)
i += 1
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.cat_m5, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(q.obs_start_mjd, q.cat_m5, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y))
ax[i].set_ylabel("estimated m5")
ax[i].grid(alpha=0.3)
i += 1
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.numGoodDiaSources_sum, linestyle='', marker=markers[b], color=colors[b], label=f"{b} sum")
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.numGoodDiaSources_median*189, linestyle='', marker='o', color='none', markeredgecolor=colors[b], label=f"{b} median*189")
ax[i].plot(q.obs_start_mjd, q.numGoodDiaSources_sum, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y), ncols=2)
ax[i].set_ylabel("Number DiaSources")
ax[i].set_ylim(-10, max(vv.numGoodDiaSources_sum.max() * 1.2, vv.numGoodDiaSources_median.max() * 189 * 1.2))
ax[i].grid(alpha=0.3)
i += 1
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.nDiaDetectors_count, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(q.obs_start_mjd, q.nDiaDetectors_count, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y))
ax[i].set_ylabel("NDetectors")
ax[i].set_ylim(0, 189)
ax[i].grid(alpha=0.3)
i += 1
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.ecliptic_lat, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(q.obs_start_mjd, q.ecliptic_lat, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y))
ax[i].set_ylabel("Ecliptic Latitude")
ax[i].grid(alpha=0.3)
i += 1
for b in q.band.unique():
    qq = q.query("band == @b")
    ax[i].plot(qq.obs_start_mjd, qq.galactic_lat, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(q.obs_start_mjd, q.galactic_lat, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(leg_x, leg_y))
ax[i].set_ylabel("Galactic Latitude")
ax[i].grid(alpha=0.3)
day_title = q.day_obs.unique()
if len(day_title) == 1:
    day_title = day_title[0]
else:
    day_title = f"{day_title.min()} - {day_title.max()}"
_ = fig.suptitle(f"DayObs {day_title}", y=0.92)